In [3]:
import sys
from matplotlib import rcParams 
rcParams["figure.figsize"] = (16,5)
sys.path.append('')

import librosa
import librosa.display

from TTS.tts.models.tacotron2 import Tacotron2 
from TTS.tts.utils import *
from TTS.tts.utils.generic_utils import setup_model
from TTS.utils.audio import AudioProcessor
from TTS.utils.io import load_config
from TTS.tts.utils.text import text_to_sequence, phoneme_to_sequence
from TTS.tts.utils.text.symbols import symbols, phonemes
import torch
from TTS.tts.utils.synthesis import synthesis

In [4]:
# Set constants

# MODEL_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_rosana_only/checkpoint_120000.pth.tar'
CONFIG_PATH =  './experiments/debug_gst_logits/config.json'

# MODEL_PATH ='/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/best_model.pth.tar'
# CONFIG_PATH = '/content/drive/MyDrive/Mestrado/TTS/GST_3speaker_CPQD/config.json'

CONFIG = load_config(CONFIG_PATH)
# CONFIG['datasets'][0]['path'] = './'
# CONFIG['output_path'] = './'
# CONFIG['audio']['signal_norm'] = False
# CONFIG['audio']['stats_path'] = ''
# CONFIG['use_phonemes'] = False
# CONFIG['save_step'] = 500
use_cuda = True

# load the model
ap = AudioProcessor(**CONFIG.audio)

num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)


# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, 2, 2, CONFIG, None)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:50.0
 | > mel_fmax:7600.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > stats_path:
 | > hop_length:256
 | > win_length:1024
 > Using model: Tacotron2
Training with 2 speakers and 2 styles
Use style target = True
Use semi supervised = True


In [6]:
# Testing CrossEntropyLoss with ignore_index
from torch import nn
criterion_style = nn.CrossEntropyLoss(ignore_index = 0)

In [ ]:
pred = torch.FloatTensor([])

In [ ]:
model

In [ ]:
model.speaker_embedding

In [ ]:
size = (1, 3)
test = torch.rand(size).type(torch.long)
model.speaker_embedding(test).shape

In [ ]:
size = (2, 50, 80)
test = torch.rand(size).type(torch.float)
enc_out , logits = model.gst_layer(test)
print(enc_out.shape, logits)

In [ ]:
enc_out.shape

In [ ]:
logits.squeeze(0).squeeze(0).squeeze(0).detach().cpu().numpy()

In [ ]:
N = CONFIG['gst']['gst_style_tokens']

feats = np.zeros((6, N))
feats

In [ ]:
feats[0] = logits.squeeze(0).squeeze(0).squeeze(0).detach().cpu().numpy()

In [ ]:
feats[0:3]

In [ ]:
import os
def vctk_direct(root_path, meta_file):
    meta_path = os.path.join(root_path, meta_file)
    
    items = []
    
    with open(meta_path, 'r', encoding= 'utf-8') as f:
        for line in f:
            cols = line.split(',')
            if(cols[1] == 'text'): # The first element is header in my file
                continue
            wav_file = cols[0]
            text = cols[1]
            speaker_name = cols[2][:-1]  # The last char is "\n" since after this line is a breakline
            items.append([text, wav_file, speaker_name])
    return items

In [ ]:
adriana = vctk_direct('./', 'adriana_expressivo_train.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('adriana_expressivo_train.csv', encoding='latin-1', sep='|')

In [ ]:
df.head()

In [ ]:
logits

In [ ]:
text_cleaner = [CONFIG.text_cleaner]
text_cleaner

In [ ]:
# s = '''Um professor, para se tornar doutor, investe, pelo menos, 8 anos de sua vida ( se partir da graduação direto pro doutorado). Se fizer graduação, mestrado e doutorado, são 10 anos. Se fizer graduação, especialização, mestrado e doutorado, lá se vão 12 anos. Sim, 12 anos de universidade. Todo esse tempo labutando com ensino, pesquisa e extensão. Produzindo, apresentando e publicando trabalhos.  Participando de debates e congressos. Dando aula. Ministrando cursos. Participando de bancas. Emitindo pareceres. Orientando artigos, projetos de conclusão de cursos, TCC's e etc, etc, etc. De repente, todo esse conhecimento arduamente produzido dentro dos parâmetros científicos e acadêmicos passa a não valer  nada ante a palavra de um esquizóide qualquer que define a forma como nós, professores, devemos conduzir o trabalho para o qual nos preparamos ao longo da vida e diuturnamente - pesquisando e estudando métodos e teorias, produzindo e submetendo nossas ideias à comunidade científica - que, todos sabemos, não é muito conhecida pela generosidade. Sem comentar os diversos perrengues -  familiares, socioemocionais, financeiros e de saúde -  que enfrentamos durante e  em virtude dessa caminhada extenuante e tão mal recompensada. Ainda assim, contra tudo e todos, persistimos cantando no mal tempo, porque temos a certeza daquilo que fazemos, e assentamos nossa autoridade e dignidade no conhecimento que produzimos. 
# Antes, a violência que se abatia sobre nós, profissionais da educação, era apenas financeira e a desvalorização social era decorrente dessa desvalorização financeira: '' Você é só uma professorazinha, ganha pouco''. 
# Agora, a violência é pior: a palavra e a autoridade intelectual do professor -  que ainda mantínhamos, apesar de tudo -  são jogados por terra. 
# Somos confrontados por pessoas que não leem, não têm conhecimento de absolutamente nada e que se informam apenas por youtubers, líderes religiosos delirantes ou figuras obscuras sem nenhum (re)conhecimento científico e que são  merecidamente proscritas do meio acadêmico.  Ainda por cima, somos chamados de '' doutrinadores'', acusados e vilipendiados das piores formas possíveis sem que os difamadores se retratem ou sofram as devidas punições por suas afirmações criminosas que têm como intuito deslegitimar, mais uma vez, a profissão docente nos violentando moralmente. E, como se não bastasse, há ''professores'' que se alinham a esse  evidente projeto de destruição da educação no país. Não me admira que a profissão esteja entre as menos procuradas pelos jovens, e que esteja, infelizmente, caminhando para a extinção.'''

import numpy as np

s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

# s = "e é"

seq = np.asarray(text_to_sequence(s, text_cleaner), dtype=np.int32)
chars_var = torch.from_numpy(seq).unsqueeze(0).cuda()

# style_wav = {'1': 0.3} # Parece um token interessante

# f = 0.1
# style_wav = {}
# n_tokens = 3
# for i in range(n_tokens):
#     style_wav[str(i)] = f

style_wav = './data/LJSpeech/LJSpeech-1.1/wavs/LJ001-0003.wav'

speaker_embedding = None
speaker_id = 2


# style_wav = None

use_cuda = True

wav, alignment, decoder_output, postnet_output, stop_tokens, _, logits = synthesis(
    model.cuda(),
    s,
    CONFIG,
    use_cuda,
    ap,
    speaker_id=speaker_id,
    speaker_embedding=speaker_embedding,
    style_wav=style_wav,
    truncated=False,
    enable_eos_bos_chars=CONFIG.enable_eos_bos_chars, #pylint: disable=unused-argument
    use_griffin_lim=True,
    do_trim_silence=False)

In [ ]:
logits[0][0][0].detach().cpu()

In [ ]:
logits.reshape(torch.tensor([0.0,0.0,1.0]).size())

In [ ]:
chars_var.size()

In [ ]:
torch.tensor([0.0,0.0,1.0])

In [ ]:
from torch.nn import functional

In [ ]:
loss = functional.binary_cross_entropy(logits.detach().cpu(), torch.tensor([0.0,0.0,1.0]))
loss

In [ ]:
alignment.shape

In [ ]:
type(logits)

In [ ]:
style_wav

In [ ]:
model.gst_layer.style_token_layer.style_tokens

In [ ]:
def text_to_seqvec(text, CONFIG):
    text_cleaner = [CONFIG.text_cleaner]
    # text ot phonemes to sequence vector
    if CONFIG.use_phonemes:
        seq = np.asarray(
            phoneme_to_sequence(text, text_cleaner, CONFIG.phoneme_language,
                                CONFIG.enable_eos_bos_chars,
                                tp=CONFIG.characters if 'characters' in CONFIG.keys() else None),
            dtype=np.int32)
    else:
        seq = np.asarray(text_to_sequence(text, text_cleaner, tp=CONFIG.characters if 'characters' in CONFIG.keys() else None), dtype=np.int32)
    return seq


def numpy_to_torch(np_array, dtype, cuda=False):
    if np_array is None:
        return None
    tensor = torch.as_tensor(np_array, dtype=dtype)
    if cuda:
        return tensor.cuda()
    return tensor

def id_to_torch(speaker_id, cuda=False):
    if speaker_id is not None:
        speaker_id = np.asarray(speaker_id)
        speaker_id = torch.from_numpy(speaker_id).unsqueeze(0)
    if cuda:
        return speaker_id.cuda().type(torch.long)
    return speaker_id.type(torch.long)

def compute_style_mel(style_wav, ap, cuda=False):
    style_mel = torch.FloatTensor(ap.melspectrogram(
        ap.load_wav(style_wav, sr=ap.sample_rate))).unsqueeze(0)
    if cuda:
        return style_mel.cuda()
    return style_mel

In [ ]:
speaker_id = id_to_torch(2, cuda=use_cuda)

In [ ]:
inputs = text_to_seqvec(s, CONFIG)
inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
inputs = inputs.unsqueeze(0)

x = model.embedding(inputs).transpose(1, 2)
x = model.encoder.inference(x)

In [ ]:
speaker_ids

In [ ]:
speaker_ids = speaker_id
gst = True
num_speakers = 3
style_mel = None

if gst:
    # B x gst_dim
    encoder_outputs, logits = model.compute_gst(x,
                                       style_mel,
                                       speaker_embeddings if model.gst_use_speaker_embedding else None)
if num_speakers > 1:
    if not model.embeddings_per_sample:
        speaker_embeddings = model.speaker_embedding(speaker_ids)[:, None]
    encoder_outputs = model._concat_speaker_embedding(encoder_outputs, speaker_embeddings)

In [ ]:
inputs = text_to_seqvec(s, CONFIG)
inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
inputs = inputs.unsqueeze(0)

model.inference(text = inputs, speaker_ids = speaker_id, style_mel = None)

In [ ]:
device = chars_var.device
device

In [ ]:
embedded_inputs = model.embedding(chars_var.type(torch.long)).transpose(1, 2)
encoder_outputs = model.encoder.inference(embedded_inputs)

In [ ]:
encoder_outputs.shape

In [ ]:
query = torch.zeros(1, 1, model.gst_embedding_dim//2).to(device)
_GST = torch.tanh(model.gst_layer.style_token_layer.style_tokens)
gst_outputs = torch.zeros(1, 1, model.gst_embedding_dim).to(device)
for k_token, v_amplifier in style_wav.items():
    print(k_token, v_amplifier)
    key = _GST[int(k_token)].unsqueeze(0).expand(1, -1, -1)
    print(query.shape, key.shape)
    gst_outputs_att, _ = model.gst_layer.style_token_layer.attention(query, key)
    print(gst_outputs_att.shape)
    gst_outputs = gst_outputs + gst_outputs_att * v_amplifier
    
gst_outputs.shape

In [ ]:
encoder_outputs = model._concat_speaker_embedding(encoder_outputs, gst_outputs)
encoder_outputs.shape

In [ ]:
gst_outputs_att.shape

In [ ]:
gst_outputs_att

In [ ]:
_

In [ ]:
for k_token, v_amplifier in style_wav.items():
    print(type(v_amplifier))

In [ ]:
model.gst_layer.style_token_layer.style_tokens.shape

In [ ]:
model.gst_layer.style_token_layer.style_tokens[0,:]*0.3

In [ ]:
model.cuda()

In [ ]:
style_mel = compute_style_mel(style_wav, ap, cuda=True)
style_mel = numpy_to_torch(style_mel, torch.float, cuda=use_cuda)

In [ ]:
style_mel.shape

In [ ]:
s = "ambulantes vendem os ingressos para a copa do mundo de dois mil e catorze , mas só alguns dias antes do dia do sorteio ."

# inputs = text_to_seqvec(s, CONFIG)
# inputs = numpy_to_torch(inputs, torch.long, cuda=use_cuda)
# inputs = inputs.unsqueeze(0)

# embedded_inputs = model.embedding(inputs).transpose(1, 2)
# encoder_outputs = model.encoder.inference(embedded_inputs)

# print(inputs.shape, encoder_outputs.shape )

encoder, logits = model.cuda().gst_layer(style_mel)

In [ ]:
logits

In [ ]:
out = torch.cat(
            torch.split(logits, 1, dim=0),
            dim=3).squeeze(0)

In [ ]:
out.shape

In [ ]:
logits.shape